In [20]:
import numpy as np
import pandas as pd
import requests

from tradingtensors.functions.utils import OandaHandler

from tradingtensors.settings.serverconfig import ID, INDICATORS_SETTINGS, RETURNS_BY_OPEN, TOKEN

import json
import datetime as dt
import time

In [21]:
TOKEN = "" #Oanda Token
ID = "" #Oanda Account ID

DEFAULT_URL ={
    'practice': 'https://api-fxpractice.oanda.com',
    'live': 'https://api-fxtrade.oanda.com'
}

DEFAULT_HEADERS = {
    'Authorization' :'Bearer ' + TOKEN
}

TIMEFRAME = 'M15'

INSTRUMENT = "EUR_USD"

MODE = 'practice'

HISTORY = 5000

DT_FORMAT='RFC3339'

currentTime = dt.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S')

DATA_PATH = 'data/'

FROM_DATE = '2010-01-01T00:00:00.000000000Z'

In [22]:
from_date = dt.datetime.strptime(FROM_DATE, '%Y-%m-%dT%H:%M:%S.000000000Z')

In [23]:
from_date

datetime.datetime(2010, 1, 1, 0, 0)

In [24]:
QUERY_URL = DEFAULT_URL[MODE] + '/v3/instruments/{}/candles'.format(INSTRUMENT)

In [25]:
QUERY_URL

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

In [26]:
#Parameters required to retrieve history
params = {
  "count": HISTORY, 
  "granularity": TIMEFRAME,
  "price": "M"
}

In [27]:
params

{'count': 5000, 'granularity': 'M15', 'price': 'M'}

In [28]:
#Headers
DEFAULT_HEADERS['Accept-Datetime-Format'] = DT_FORMAT

In [29]:
keepLoop = True
firsLoop = True
history = []

while keepLoop:
    
    #Make API call repeated 
    try:
        response = requests.get(QUERY_URL, headers=DEFAULT_HEADERS, params=params).json()
        #with open('data/history.1.json') as f:
        #    response = json.load(f)
        candles = response["candles"]
        
        if not history :
            history = response
        else :
            for x in reversed(candles) : 
                history['candles'].insert(0, x)
            
        params["to"] = candles[0]['time']
        
        firstCandlesDate = dt.datetime.strptime(candles[0]['time'], '%Y-%m-%dT%H:%M:%S.000000000Z')
        
        if firstCandlesDate <= from_date:
            keepLoop = False
        
        

    except (RequestException, KeyError) as e:
        print ("Failed to retrieve instrument history from Oanda, {}".format(e))


In [30]:
with open(DATA_PATH+INSTRUMENT+'_'+TIMEFRAME+'_history.json'.format(currentTime), 'w') as outfile:
    json.dump(history, outfile)

In [139]:
history['candles']

[{'complete': True,
  'volume': 1402,
  'time': '2018-07-13T18:00:00.000000000Z',
  'mid': {'o': '21.60172', 'h': '21.60538', 'l': '21.58113', 'c': '21.58250'}},
 {'complete': True,
  'volume': 979,
  'time': '2018-07-13T19:00:00.000000000Z',
  'mid': {'o': '21.58226', 'h': '21.59988', 'l': '21.57230', 'c': '21.58914'}},
 {'complete': True,
  'volume': 320,
  'time': '2018-07-13T20:00:00.000000000Z',
  'mid': {'o': '21.59188', 'h': '21.60720', 'l': '21.58643', 'c': '21.59504'}},
 {'complete': True,
  'volume': 605,
  'time': '2018-07-15T21:00:00.000000000Z',
  'mid': {'o': '1.16908', 'h': '1.16934', 'l': '1.16817', 'c': '1.16856'}},
 {'complete': False,
  'volume': 617,
  'time': '2018-07-15T22:00:00.000000000Z',
  'mid': {'o': '1.16854', 'h': '1.16856', 'l': '1.16784', 'c': '1.16822'}}]